In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('./train/train.csv')
test = pd.read_csv('./test/test.csv')

train.drop(['Name', 'Description', 'PetID', 'RescuerID'], axis=1, inplace=True)
test.drop(['Name', 'Description', 'PetID', 'RescuerID'], axis=1, inplace=True)

target = train['AdoptionSpeed']
train.drop(['AdoptionSpeed'], axis=1, inplace=True)

# StratifiedJFold for Classification

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

k_fold = 5
kf = StratifiedKFold(n_splits=k_fold, random_state=42, shuffle=True)
fold_splits = kf.split(train, target)

pred_train = np.zeros((train.shape[0])) # train 관측치 수
pred_test_ = np.zeros((test.shape[0],5)) # test 관측치 수, 범주의 수 # regression이면 범주의 수 없이 ㄱㄱ

i = 1
for dev_index, val_index in fold_splits: # 알아서 나눠줌
    dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
    dev_y, val_y = target[dev_index], target[val_index]
    
    model = DecisionTreeClassifier()
    model.fit(dev_X, dev_y)
    pred_val_y = model.predict(val_X)
    pred_test_y = model.predict_proba(test)

    pred_train[val_index] = pred_val_y
    pred_test_ = pred_test_ + pred_test_y
    i = i +1
pred_test_ = pred_test_ / 5.0

In [4]:
pred_train

array([2., 3., 3., ..., 2., 4., 3.])

In [5]:
np.argmax(pred_test_, axis=1)

array([1, 2, 2, ..., 1, 3, 4], dtype=int64)